In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import torch.nn.functional as F

## 4. Back Propogation (cont.)

Instead of using our self implemented gradient() or back_prop() functions, we can use PyTorch's library functions for very similar results.

In [2]:
# Training Data
x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

w = Variable(torch.Tensor([1.0]), requires_grad=True) #any random value

In [3]:
# Forward and Loss functions from Part_1
def forward(w, x):
    return(w*x)

def loss(w, x, y):
    y_pred = forward(w,x)
    loss = (y_pred - y) * (y_pred - y)
    return(loss)

In [4]:
# Returns a tensor w*x
forward(w, 4).data[0] #w is a tensor

tensor(4.)

In [5]:
# Train forward model using built-in PyTorch Tensor library
# Tensor stores computational graph automatically since requires_grad=True
for epoch in range(10):
    for x_val, y_val in zip(x_data, y_data):
        l = loss(w, x_val, y_val)
        l.backward()  #knows to go backward since the computations were stored in tensor, w
        w.data = w.data - 0.01*w.grad.data
        
        w.grad.data.zero_() #erase records of previous passes
    print(f'Epoch: {epoch}, Loss = {l}')
print(f'Final weight, w = {w.data[0]}')

Epoch: 0, Loss = tensor([7.3159], grad_fn=<MulBackward0>)
Epoch: 1, Loss = tensor([3.9988], grad_fn=<MulBackward0>)
Epoch: 2, Loss = tensor([2.1857], grad_fn=<MulBackward0>)
Epoch: 3, Loss = tensor([1.1946], grad_fn=<MulBackward0>)
Epoch: 4, Loss = tensor([0.6530], grad_fn=<MulBackward0>)
Epoch: 5, Loss = tensor([0.3569], grad_fn=<MulBackward0>)
Epoch: 6, Loss = tensor([0.1951], grad_fn=<MulBackward0>)
Epoch: 7, Loss = tensor([0.1066], grad_fn=<MulBackward0>)
Epoch: 8, Loss = tensor([0.0583], grad_fn=<MulBackward0>)
Epoch: 9, Loss = tensor([0.0319], grad_fn=<MulBackward0>)
Final weight, w = 1.9512161016464233


Remember that we really just need the input variables, weights, and the gradient so that we can calculate a direction of minimizing loss.

## 5. Linear Regression in PyTorch

We want to go further than just the simplest model. Now, we want to:
1. Design model with Variables  
2. Construct Loss and Optimizer (from PyTorch)  
3. Train Model  
4. Test Model

In [6]:
# Training data
x_data = Variable(torch.Tensor([[1.0], [2.0], [3.0]]))
y_data = Variable(torch.Tensor([[2.0], [4.0], [6.0]]))

__1. Design Model__

In [7]:
# Linear Model class
class Model(torch.nn.Module):

    def __init__(self):
        """ Instantiate two nn.Linear modules """
        super(Model, self).__init__()
        self.linear = torch.nn.Linear(1,1) #linear model with 1 input, 1 output
    
    def forward(self, x):
        """ Predict function using model after training
        Param x :: Variable
        Return y_pred :: Variable
        """
        y_pred = self.linear(x) #PyTorch's linear prediction
        return(y_pred)

__2. Construct Loss and Optimizer__

In [8]:
# Model, MSE Loss Function, SGD Optimizer
model = Model()
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

__3. Train Model__

In [9]:
# Train model
counter = 0
for epoch in range(500):
    y_pred = model(x_data) #randomly decides starting weight, w on first pass
    loss = criterion(y_pred, y_data)
    
    if counter < 10:
        print(f'Epoch: {epoch}, Loss = {loss.item()}')
        counter+=1
        
    optimizer.zero_grad() #initialize gradient
    loss.backward()       #perform backward pass
    optimizer.step()      #update the variable (parameters)

Epoch: 0, Loss = 34.4931640625
Epoch: 1, Loss = 15.360153198242188
Epoch: 2, Loss = 6.842611312866211
Epoch: 3, Loss = 3.050776481628418
Epoch: 4, Loss = 1.3626923561096191
Epoch: 5, Loss = 0.6111398339271545
Epoch: 6, Loss = 0.2765045464038849
Epoch: 7, Loss = 0.12747128307819366
Epoch: 8, Loss = 0.06106254458427429
Epoch: 9, Loss = 0.03143744543194771


__4. Test Model__

In [10]:
# Test model
x_test = Variable(torch.Tensor([[4.0]]))
y_test = model.forward(x_test).data[0][0]
print(f'Input = {x_test.data[0][0]}: Output = {y_test}')

Input = 4.0: Output = 7.997081756591797


## 6. Logistic

Binary classification using Logistic Regression, a common building block of neural networks.
  
Logistic Regression is very similar to linear regression; we simply add a sigmoid function at the end of our forward pass to get value from 0 to 1. We round this value to get a final output.
  
Because of this sigmoid function, we change our loss function to Cross Entropy Loss.

__Model__

In [11]:
# Logistic Regression
from IPython.display import Math
Math(r'x \rightarrow Linear \rightarrow  Sigmoid \rightarrow \hat{y}')

<IPython.core.display.Math object>

In [12]:
# Forward pass
def forward(w, x, b):
    return(w*x + b)

def sigmoid(Z):
    d = np.array([1+np.exp(-z) for z in Z])
    return(1/d)

In [13]:
# Cross Entropy Loss
def CE_Loss(yhat_, y_, p=False):
    assert len(yhat_) == len(y_)
    loss_list = []
    N = len(yhat_)
    for i in range(N):
        y = y_[i]
        yhat = yhat_[i]

        l = -1* (y*np.log(yhat) + (1-y)*np.log(1-yhat))
        loss_list.append(l)
        
        if p:
            print(l)
    # Total loss
    L = np.sum(loss_list) / N
    return(L)

__Example__  
  
  Let us try on some data. We can see that as our y_pred values are closer to the true binary y value, our loss is smaller.

In [14]:
# Loss on sample data
y_pred = [0.2, 0.8, 0.1, 0.9]
y = [1, 1, 0, 0]
CE_Loss(y_pred, y, True)

1.6094379124341003
0.2231435513142097
0.10536051565782628
2.302585092994046


1.0601317681000455

__PyTorch__

In [18]:
# Linear Model class with Sigmoid
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.linear = torch.nn.Linear(1, 1) #still linear model with 1 input, 1 output
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))  #wrap foward pass with sigmoid
        return(y_pred)

In [19]:
# Model, Loss, Optimizer
model = Model()
criterion = torch.nn.BCELoss()            #binary CE Loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  #SGD optimizer, same as before

In [20]:
# Training data
x_data = Variable(torch.Tensor([[1.0], [2.0], [3.0], [4.0]]))
y_data = Variable(torch.Tensor([[0.], [0.], [1.], [1.]]))

# Train Model
for epoch in range(1000):
    y_pred = model(x_data)
    
    loss = criterion(y_pred, y_data)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Test Model
x_test = Variable(torch.Tensor([[1.0], [7.0]]))
y_test = model.forward(x_test).data
print('X Input:',x_test)
print('Y Output:', y_test)

X Input: tensor([[1.],
        [7.]])
Y Output: tensor([[0.2981],
        [0.9880]])


We can see here that if we round our test's output values, we would have gotten the answer expect.